In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import numpy as np
from bs4 import BeautifulSoup

# Set the path to the Microsoft Edge driver
PATH = "C:/Users/ihp/Downloads/edgedriver_win64/msedgedriver.exe"
service = Service(PATH)

# Initialize lists to store data
URL, JobTitle, Company, Ratings, Reviews, Experience, Salary, Location, Job_Posted, Skills = [], [], [], [], [], [], [], [], [], []

# URL of the job search page
url = "https://www.naukri.com/a-b-c-d-e-f-g-h-i-kj-k-l-m-n-o-p-q-a-s-t-u-v-w-x-y-z-1-2-3-4-5-6-7-8-9-0-jobs-6350?k=a%2C%20b%2C%20c%2C%20d%2C%20e%2C%20f%2C%20g%2C%20h%2C%20i%2C%20kj%2C%20k%2C%20l%2C%20m%2C%20n%2C%20o%2C%20p%2C%20q%2C%20a%2C%20s%2C%20t%2C%20u%2C%20v%2C%20w%2C%20x%2C%20y%2C%20z%2C%201%2C%202%2C%203%2C%204%2C%205%2C%206%2C%207%2C%208%2C%209%2C%200"

# Initialize the Edge WebDriver
driver = webdriver.Edge(service=service)
driver.get(url)

try:
    # Wait for and click the privacy policy button if it's present
    button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#root > div.search-result-container > div.privacyPolicy > div > button')))
    button.click()
except:
    print("Button is not present")
    print(f'webdriver wait failed for {url}')
    time.sleep(6)

while True:
    try:
        # Wait for the job listings to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'jobTuple')))

        # Parse the page with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'lxml')
        articles = soup.find_all('article', class_='jobTuple')
        for i in articles:
            # Job_URL
            url = i.find('a', class_="title ellipsis")
            if url is not None:
                ur = url.get('href')
                print(ur)
                URL.append(ur)
            else:
                URL.append(np.nan)

            # Job Role or title
            title = i.find('a', class_="title ellipsis")
            if title is not None:
                tit = title.text
                JobTitle.append(tit)
            else:
                JobTitle.append(np.nan)

            # Company name
            name = i.find('a', class_='subTitle ellipsis fleft')
            if name is not None:
                nam = name.text
                Company.append(nam)
            else:
                Company.append(np.nan)

            # Ratings
            ratings = i.find('span', class_="starRating fleft")
            if ratings is not None:
                rat = ratings.text
                Ratings.append(rat)
            else:
                Ratings.append(np.nan)

            # Review count
            reviews = i.find('span', class_="reviewsCount fleft")
            if reviews is not None:
                rev = reviews.text
                Reviews.append(rev)
            else:
                Reviews.append(np.nan)

            # Years of experience
            years = i.find('span', class_="ellipsis fleft expwdth")
            if years is not None:
                year = years.text
                Experience.append(year)
            else:
                Experience.append(np.nan)

            # Salary offered
            salary = i.find('span', class_="ellipsis fleft")
            if salary is not None:
                sal = salary.text
                Salary.append(sal)
            else:
                Salary.append(np.nan)

            # Location
            location = i.find('span', class_="ellipsis fleft locWdth")
            if location is not None:
                loc = location.text
                Location.append(loc)
            else:
                Location.append(np.nan)

            # Number of days since job posted
            days = i.find('span', class_="fleft postedDate")
            if days is not None:
                day = days.text
                Job_Posted.append(day)
            else:
                Job_Posted.append(np.nan)

            # Skillset Required
            skills = i.find('ul', class_="tags has-description")
            if skills:
                skillset = ", ".join(skill.text.strip() for skill in skills.find_all("li"))
                Skills.append(skillset)
            else:
                Skills.append(np.nan)

        try:
            # Click the next button to go to the next page of job listings
            next_button = driver.find_element(By.CSS_SELECTOR, "#root > div.search-result-container > div.search-result-wrap > div > section.listContainer.fleft > div.mt-36.mb-54 > div > a.fright.fs14.btn-secondary.br2")
            next_button.click()
            time.sleep(2)
        except:
            break  # Exit the loop if there's no next button

    except Exception as e:
        print(f"Error: {str(e)}")
        driver.refresh()  # Reload the page if an exception occurs
        continue  # Continue to the next iteration of the loop if there's an error

# Close the WebDriver when done
driver.quit()


In [ ]:
# After the loop, you can create a DataFrame to store the data
data = {
    'URL': URL,
    'JobTitle': JobTitle,
    'Company': Company,
    'Ratings': Ratings,
    'Reviews': Reviews,
    'Experience': Experience,
    'Salary': Salary,
    'Location': Location,
    'Job_Posted': Job_Posted,
    'Skills': Skills
}

df = pd.DataFrame(data)
df.to_csv('Naukri_job_data3.csv', index=False)